# Rubin TAP + Python Utilities Guide
**Author:** McAelan Remigio

**Based on original material by:** Leanne Guy, Melissa Graham
**Last updated:** June 6, 2025
**Credit:** This simplified guide is derived from the original Rubin Science Platform tutorial by Leanne Guy, created in the context of Rubin DP0.1. It aims to reduce complexity and improve clarity for new learners.

This guide introduces the Rubin Science Platform TAP service through:
- ADQL querying (basic and aggregated)
- When to use search() vs submit_job()
- Basic matplotlib histograms
- NaN handling and pandas techniques

Target audience: Astronomy & data science learners with minimal programming experience.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps
from lsst.rsp import get_tap_service

## TAP Service Connection

We begin by connecting to the Rubin TAP service. This gives us access to the LSST data tables via ADQL, known as the Astronomical Data Query Language that is a SQL dialact for astronomy.

In [ ]:
service = get_tap_service("ssotap")
assert service is not None

## Schema Inspection: MPCORB Table

We can inspect the columns, datatypes, and descriptions of the `dp03_catalogs_10yr.MPCORB` table using TAP metadata.


In [ ]:
results = service.search("""
SELECT column_name, datatype, description, unit 
FROM TAP_SCHEMA.columns 
WHERE table_name = 'dp03_catalogs_10yr.MPCORB'
""")
print(results.to_table().to_pandas())

## Basic ADQL Query: Min/Max Object IDs

Let’s look at the smallest and largest object IDs to get a sense of range.

In [ ]:
results = service.search("""
SELECT MIN(ssObjectId), MAX(ssObjectId) 
FROM dp03_catalogs_10yr.MPCORB
""")
print(results.to_table())

## Aggregation Example: Summary Statistics

We'll compute some basic statistics: count, min, max, and average values of key orbital elements.

In [ ]:
query = """
SELECT COUNT(*) AS total,
       MIN(a) AS min_a,
       MAX(a) AS max_a,
       AVG(e) AS avg_e
FROM dp03_catalogs_10yr.MPCORB
"""

job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
assert job.phase == 'COMPLETED'
df = job.fetch_result().to_table().to_pandas()
print(df)

## Visualization: Histogram of Eccentricity

We’ll simulate a small dataset for visualization. Histograms are log-scaled to highlight distributions over wide ranges.

In [ ]:
df = pd.DataFrame({'e': np.random.beta(2, 5, 10000)})

fig, ax = plt.subplots(2, 3, figsize=(10, 6), sharey=False)
ax[0, 0].hist(df['e'], bins=100, log=True)
ax[0, 0].set_xlabel('Eccentricity')
ax[0, 0].set_ylabel('log(Number)')
fig.suptitle('Histograms for Key Orbital Elements')
fig.tight_layout()
plt.show()

## Data Cleaning: Handling NaN Values

We'll remove any rows that have missing (`NaN`) values in key columns, then reset the index to keep the dataframe tidy.

In [ ]:
df_cleaned = df.dropna(subset=['e'])  # Copy version
df.dropna(subset=['e'], inplace=True) # In-place version
df.reset_index(drop=True, inplace=True)

print('Number of rows after dropping NaNs:', len(df))